"""
Generate data with specified changepoints
Compare ELBO fits for changepoint model with different
numbers of changepoints
"""

In [2]:
import os
os.environ['OMP_NUM_THREADS'] = '1'

import pymc3 as pm
import theano.tensor as tt
from scipy import stats
import numpy as np
import pylab as plt
import sys
sys.path.append('/media/bigdata/firing_space_plot/ephys_data')
from time import time

sys.path.append('/media/bigdata/projects/parametric_firing/src')
from hierarchical_fake_firing import fake_poisson_firing
from poisson_changepoint_models import (return_unpooled_model, 
                                        return_pooled_model,
                                        return_hierarchical_model,
                                        fit_model,)

In [15]:
def firing_properties(data_array):
    # average for whole dataset
    mean_total_firing = data_array.mean(axis=None)
    # max average firing for a neuron
    max_neuron_firing = np.max(data_array.mean(axis=(0,2)))
    return mean_total_firing, max_neuron_firing

In [10]:
# Params to test:
# 1) neuron count
# 2) trials count
# 3) states count
# 4) emission rate
# 5) emission jitter

firing_params = dict(
        n_nrns = 7,
        n_trials = 15,
        n_states = 3,
        duration = 1000,
        min_duration = 200,
        ceil_p = 0.1,
        jitter_p = 0.2,
        bin_size = 25
        )

fit_params = dict(
        n_fit = int(1e5),
        n_samples = 2000
)

data_array, true_r, true_tau, state_inds, trial_p = fake_poisson_firing(**firing_params)

In [11]:
model_gen_funcs = [return_unpooled_model,
                  return_pooled_model,
                  return_hierarchical_model]
model_params = [data_array, firing_params['n_states']]
model_names = [x.__name__[7:] for x in model_gen_funcs]
model_list = [f(*model_params) for f in model_gen_funcs]

In [12]:
time_list = []
trace_list = []

for model in model_list:
    start_t = time()
    trace, approx = fit_model(model, **fit_params)
    trace_list.append(trace)
    time_taken = time() - start_t
    time_list.append(time_taken)
    
print(time_list)

Finished [100%]: Average Loss = 4,441.2


Finished [100%]: Average Loss = 3,864.5


Finished [100%]: Average Loss = 4,030.8


[63.507073163986206, 41.43665051460266, 66.8029396533966]


In [13]:
mode_tau_list = []
for trace in trace_list:
    tau = trace['tau']
    mode_tau = stats.mode(np.vectorize(np.int)(tau),axis=0)[0][0]
    mode_tau_list.append(mode_tau)
tau_errors = [np.abs(true_tau - x).flatten() for x in mode_tau_list]

In [49]:
model_dict = {f'{this_name}' : dict(error = this_error, runtime = this_time) \
              for this_name, this_error, this_time \
              in zip(model_names, tau_errors, time_list)}
out_dict = dict(
    **dict(
        zip(
            ['mean_total_firing', 'max_neuron_firing'], 
            firing_properties(data_array)
        )
    ),
    **model_dict,
    **firing_params,
)

In [50]:
out_dict

{'mean_total_firing': 0.6630952380952381,
 'max_neuron_firing': 1.6733333333333333,
 'unpooled_model': {'error': array([16,  1, 10,  1, 12,  1, 16,  0, 13,  3, 16,  0, 16,  3,  7,  1, 19,
          1, 15,  1,  8,  2, 15,  3, 16,  6, 14,  2, 16,  0]),
  'runtime': 63.507073163986206},
 'pooled_model': {'error': array([1, 1, 1, 2, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 2, 1, 1, 1, 1, 1, 2, 0,
         1, 2, 1, 0, 4, 1, 0, 1]),
  'runtime': 41.43665051460266},
 'hierarchical_model': {'error': array([13,  0,  2,  2, 10,  0,  2,  0,  2,  1,  3,  0,  1,  1,  1,  1, 10,
          1,  8,  1,  1,  0, 11,  2, 10,  1, 10,  1,  6,  1]),
  'runtime': 66.8029396533966},
 'n_nrns': 7,
 'n_trials': 15,
 'n_states': 3,
 'duration': 1000,
 'min_duration': 200,
 'ceil_p': 0.1,
 'jitter_p': 0.2,
 'bin_size': 25}